In [3]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Exploratory Data Analysis (EDA) for Benin Solar Dataset\n",
    "\n",
    "This notebook performs EDA on the Benin solar dataset, including summary statistics, missing value analysis, outlier detection, cleaning, time series analysis, and visualizations as per Task 2 requirements."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": '',
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy.stats import zscore\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "%matplotlib inline\n",
    "\n",
    "# Load the dataset\n",
    "df = pd.read_csv('data/benin.csv')\n",
    "\n",
    "# Convert Timestamp to datetime\n",
    "df['Timestamp'] = pd.to_datetime(df['Timestamp'])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Summary Statistics & Missing Values"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": '',
   "metadata": {},
   "outputs": [],
   "source": [
    "# Summary statistics\n",
    "print('Summary Statistics:')\n",
    "print(df.describe())\n",
    "\n",
    "# Missing values\n",
    "print('\\nMissing Values:')\n",
    "missing = df.isna().sum()\n",
    "print(missing)\n",
    "\n",
    "# Columns with >5% missing\n",
    "print('\\nColumns with >5% Missing Values:')\n",
    "missing_pct = df.isna().mean()\n",
    "print(missing_pct[missing_pct > 0.05])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Outlier Detection & Cleaning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": '',
   "metadata": {},
   "outputs": [],
   "source": [
    "# Z-scores for key columns\n",
    "key_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']\n",
    "z_scores = df[key_cols].apply(zscore, nan_policy='omit')\n",
    "outliers = (z_scores.abs() > 3).any(axis=1)\n",
    "print(f'Number of outlier rows (|Z|>3): {outliers.sum()}')\n",
    "\n",
    "# Impute missing values with median for key columns\n",
    "for col in key_cols:\n",
    "    df[col].fillna(df[col].median(), inplace=True)\n",
    "\n",
    "# Drop remaining rows with missing values in critical columns\n",
    "df_clean = df.dropna(subset=['GHI', 'DNI', 'DHI'])\n",
    "\n",
    "# Export cleaned data\n",
    "df_clean.to_csv('data/benin_clean.csv', index=False)\n",
    "print('Cleaned data saved to data/benin_clean.csv')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Time Series Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": '',
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot GHI, DNI, DHI, Tamb vs. Timestamp\n",
    "plt.figure(figsize=(12, 8))\n",
    "plt.plot(df_clean['Timestamp'], df_clean['GHI'], label='GHI')\n",
    "plt.plot(df_clean['Timestamp'], df_clean['DNI'], label='DNI')\n",
    "plt.plot(df_clean['Timestamp'], df_clean['DHI'], label='DHI')\n",
    "plt.plot(df_clean['Timestamp'], df_clean['Tamb'], label='Tamb')\n",
    "plt.title('Time Series of Solar Irradiance and Temperature (Benin)')\n",
    "plt.xlabel('Timestamp')\n",
    "plt.ylabel('Value')\n",
    "plt.legend()\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Monthly averages\n",
    "df_clean['Month'] = df_clean['Timestamp'].dt.month\n",
    "monthly_avg = df_clean.groupby('Month')[['GHI', 'DNI', 'DHI']].mean()\n",
    "monthly_avg.plot(kind='bar', figsize=(10, 6))\n",
    "plt.title('Monthly Average GHI, DNI, DHI')\n",
    "plt.xlabel('Month')\n",
    "plt.ylabel('Average Value')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Cleaning Impact"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": '',
   "metadata": {},
   "outputs": [],
   "source": [
    "# Group by Cleaning flag and plot ModA, ModB\n",
    "if 'Cleaning' in df_clean.columns:\n",
    "    cleaning_impact = df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean()\n",
    "    cleaning_impact.plot(kind='bar', figsize=(8, 6))\n",
    "    plt.title('Average ModA and ModB by Cleaning Status')\n",
    "    plt.xlabel('Cleaning Status')\n",
    "    plt.ylabel('Average Value')\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Correlation & Relationship Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": '',
   "metadata": {},
   "outputs": [],
   "source": [
    "# Correlation heatmap\n",
    "corr_cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']\n",
    "plt.figure(figsize=(8, 6))\n",
    "sns.heatmap(df_clean[corr_cols].corr(), annot=True, cmap='coolwarm')\n",
    "plt.title('Correlation Heatmap')\n",
    "plt.show()\n",
    "\n",
    "# Scatter plots\n",
    "plt.figure(figsize=(12, 4))\n",
    "plt.subplot(1, 3, 1)\n",
    "plt.scatter(df_clean['WS'], df_clean['GHI'], alpha=0.5)\n",
    "plt.title('WS vs. GHI')\n",
    "plt.xlabel('Wind Speed')\n",
    "plt.ylabel('GHI')\n",
    "\n",
    "plt.subplot(1, 3, 2)\n",
    "plt.scatter(df_clean['RH'], df_clean['Tamb'], alpha=0.5)\n",
    "plt.title('RH vs. Tamb')\n",
    "plt.xlabel('Relative Humidity')\n",
    "plt.ylabel('Ambient Temperature')\n",
    "\n",
    "plt.subplot(1, 3, 3)\n",
    "plt.scatter(df_clean['RH'], df_clean['GHI'], alpha=0.5)\n",
    "plt.title('RH vs. GHI')\n",
    "plt.xlabel('Relative Humidity')\n",
    "plt.ylabel('GHI')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Wind & Distribution Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count":'',
   "metadata": {},
   "outputs": [],
   "source": [
    "# Simple wind rose (bar plot for WD and WS)\n",
    "plt.figure(figsize=(8, 6))\n",
    "plt.hist(df_clean['WD'], bins=36, weights=df_clean['WS'], edgecolor='black')\n",
    "plt.title('Wind Rose (Wind Speed by Direction)')\n",
    "plt.xlabel('Wind Direction (degrees)')\n",
    "plt.ylabel('Wind Speed')\n",
    "plt.show()\n",
    "\n",
    "# Histograms for GHI and WS\n",
    "plt.figure(figsize=(12, 4))\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.hist(df_clean['GHI'], bins=30, edgecolor='black')\n",
    "plt.title('GHI Distribution')\n",
    "plt.xlabel('GHI')\n",
    "plt.ylabel('Frequency')\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.hist(df_clean['WS'], bins=30, edgecolor='black')\n",
    "plt.title('Wind Speed Distribution')\n",
    "plt.xlabel('Wind Speed')\n",
    "plt.ylabel('Frequency')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Temperature Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": '',
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze RH impact on Tamb and GHI\n",
    "print('Correlation between RH and Tamb:', df_clean['RH'].corr(df_clean['Tamb']))\n",
    "print('Correlation between RH and GHI:', df_clean['RH'].corr(df_clean['GHI']))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Bubble Chart"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": '',
   "metadata": {},
   "outputs": [],
   "source": [
    "# Bubble chart: GHI vs. Tamb, size = RH\n",
    "plt.figure(figsize=(10, 6))\n",
    "plt.scatter(df_clean['Tamb'], df_clean['GHI'], s=df_clean['RH']*10, alpha=0.5)\n",
    "plt.title('GHI vs. Tamb (Bubble Size = RH)')\n",
    "plt.xlabel('Ambient Temperature')\n",
    "plt.ylabel('GHI')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Key Observations\n",
    "- **Missing Values**: Columns with >5% missing values (if any) were imputed with median to preserve data integrity.\n",
    "- **Outliers**: Identified and retained outliers (|Z|>3) unless critical for analysis; cleaning improved data quality.\n",
    "- **Patterns**: GHI and DNI show strong daily peaks, with potential seasonal trends observed in monthly averages."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Exploratory Data Analysis (EDA) for Benin Solar Dataset\n',
    '\n',
    'This notebook performs EDA on the Benin solar dataset, including summary statistics, missing value analysis, outlier detection, cleaning, time series analysis, and visualizations as per Task 2 requirements.']},
  {'cell_type': 'code',
   'execution_count': '',
   'metadata': {},
   'outputs': [],
   'source': ['import pandas as pd\n',
    'import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import seaborn as sns\n',
    'from scipy.stats import zscore\n',
    'import warnings\n',
    "warnings.filterwarnings('ignore')\n",
    '%matplotlib inline\n',
    '\n',
    '# Load the dataset\n',
    "df = pd.read_csv('data/benin.csv')\n",
    '\n',
    '# Convert Timestamp to datetime\n',
    "df['Timestamp'] = pd.to_datetime(df['Timestamp'])"]},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': ['## 1. Summary Statistics 